## Test Swedish PM SOU documents and sent it to SDG mapper
version 0.1
* This [Jupyter Notebook](https://github.com/salgo60/SDGMapperSweden/blob/main/Notebook/SDGMapper%20Swedish%20PM%20document%20SOU.ipynb)  / GITHUB [SDGMapperSweden](https://github.com/salgo60/SDGMapperSweden)


* [Create query at the Swedish PM](http://data.riksdagen.se/dokumentlista/)   


### Test "Vägen till en klimatpositiv framtid" SOU 2020:4 H8B34
* [text](http://data.riksdagen.se/dokument/H8B34.text) / [html](http://data.riksdagen.se/dokument/H8B34.html) /  [json](http://data.riksdagen.se/dokumentlista/?sok=H8B34&utformat=json) 


## Version history ##
* 0.1 
* 0.2 looks we have a size problem of SOU documents see [#4](https://github.com/salgo60/SDGMapperSweden/issues/4)
* 0.3 changed Exception what is printed see [#4](https://github.com/salgo60/SDGMapperSweden/issues/4) also tracked size of Swedish PM [SOU documents](https://en.wikipedia.org/wiki/Statens_offentliga_utredningar)

In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", datetime.now())

Last run:  2022-06-21 12:41:42.382993


In [2]:
class RenderHyperlink(object):
    def __init__(self, key, link, *args):
        link = link + "/" if not link.endswith("/") else link
        
        for arg in args:
            link += arg + "/"         
        self.__url = "<a href={}>{}</a>".format(link, key)
    
    def __repr__(self):
        from IPython.core.display import display, HTML
        display(HTML(self.__url))
        return "" 


In [3]:
# Document text from html
from bs4 import BeautifulSoup
import requests,time,json
tracksizeProblem = 1000000
tracksizeOk = 0
trackbiggestSOUdocument = 0 

def SDGMapperHTML(urlhtml):
    global tracksizeProblem
    global tracksizeOk
    global trackbiggestSOUdocument
    req = requests.get(urlhtml)
    
    #jsondata = json.loads(doc_json)
    #doc_txt = jsondata["dokumentstatus"]["dokument"]["text"]
    soup = BeautifulSoup(req.content, 'html.parser')
    #print(soup)
    doc_txt = soup.get_text()
    #print(doc_txt)
    print("Length text:\t",len(doc_txt))
    if len(doc_txt) > trackbiggestSOUdocument:
        trackbiggestSOUdocument = len(doc_txt) 

    try:
        if len(doc_txt):
            urlSDGMapper = "https://knowsdgs.jrc.ec.europa.eu/"
            s = requests.Session()
            tkn = s.post(urlSDGMapper + "session/token")
            time.sleep(6)

            myKey = "db0c87cbdf761defc71ea93010fdde41"

            headers = {"Content-Type": "application/json", "X-Csrf-Token": tkn.text, "X-Api-Key": myKey }
            rSDG = s.post(urlSDGMapper + "api/rest/mappingdata", headers=headers, json={"input_text": doc_txt})
            print(rSDG)
            jsonSDG = json.loads(rSDG.text)
            print(jsonSDG)
            if len(doc_txt) > tracksizeOk:
                tracksizeOk = len(doc_txt)
                print("new ok higher size ",tracksizeOk)            

        else:
            print("\t No text")
    except Exception as e: 
        print("Exception:\t",e)
        print(rSDG.text)

        if len(doc_txt) < tracksizeProblem:          
            tracksizeProblem = len(doc_txt)
            print("new error lower size ",tracksizeProblem)            
    return 
#SDGMapper("This is a test text")

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Appended depreceated 

import pandas
import urllib3, json
import pandas as pd 
from tqdm.notebook import trange  
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
urlbase ="http://data.riksdagen.se/dokumentlista/?sok=&doktyp=sou&rm=&from=&tom=&ts=&bet=&tempbet=&nr=&org=&iid=&webbtv=&talare=&exakt=&planering=&sort=rel&sortorder=desc&rapport=&utformat=json&a=s&start="


dftot = pd.DataFrame()
for i in trange(1,215): 
#for i in trange(1,10): 
    url = urlbase + str(i)
    r = http.request('GET', url)
    data = json.loads(r.data)
    r = http.request('GET', url)
    dftot = dftot.append(pd.DataFrame(data["dokumentlista"]["dokument"]),sort=False)
dftot.head()


  0%|          | 0/9 [00:00<?, ?it/s]

,traff,domain,database,datum,id,rdrest,slutdatum,rddata,plats,klockslag,publicerad,systemdatum,undertitel,kalla,kall_id,dok_id,dokumentformat,dokument_url_text,dokument_url_html,inlamnad,motionstid,tilldelat,lang,url,relurl,titel,rm,organ,relaterat_id,doktyp,typ,subtyp,beteckning,tempbeteckning,nummer,status,score,sokdata,summary,notisrubrik,notis,dokintressent,filbilaga,avdelning,struktur,audio,video,debattgrupp,debattdag,beslutsdag,beredningsdag,justeringsdag,beslutad,debattsekunder,ardometyp,reservationer,debatt,debattnamn,dokumentnamn
0,1,rdwebb,dokument,2022-01-28,HAB33,None,,None,,,2022-01-28,2022-02-03 14:46:31,,,,HAB33,,//data.riksdagen.se/dokument/HAB33.text,//data.riksdagen.se/dokument/HAB33.html,,,,,,,Sveriges tillgång till vaccin mot covid-19 - f...,2022,,,sou,sou,,3,,3,,276,{'titel': 'Sveriges tillgång till vaccin mot c...,sou 2022 3 Sveriges tillgång till vaccin mot ...,Sveriges tillgång till vaccin mot covid-19 - f...,sou 2022 3 Sveriges tillgång till vaccin mot ...,None,"{'fil': [{'typ': 'pdf', 'namn': 'sou_2022__3.p...",dokument,,,,,,,,,,,,,None,Statens offentliga utredningar,Statens offentliga utredningar
1,2,rdwebb,dokument,2022-02-02,HAB32,None,,None,,,2022-02-02,2022-02-03 14:45:32,,,,HAB32,,//data.riksdagen.se/dokument/HAB32.text,//data.riksdagen.se/dokument/HAB32.html,,,,,,,En skärpt syn på brott mot journalister och ut...,2022,,,sou,sou,,2,,2,,276,{'titel': 'En skärpt syn på brott mot journali...,sou 2022 2 En skärpt syn på brott mot journal...,En skärpt syn på brott mot journalister och ut...,sou 2022 2 En skärpt syn på brott mot journal...,None,"{'fil': [{'typ': 'pdf', 'namn': 'sou_2022__2.p...",dokument,,,,,,,,,,,,,None,Statens offentliga utredningar,Statens offentliga utredningar
2,3,rdwebb,dokument,2022-01-26,HAB31,None,,None,,,2022-01-26,2022-01-26 12:47:51,,,,HAB31,,//data.riksdagen.se/dokument/HAB31.text,//data.riksdagen.se/dokument/HAB31.html,,,,,,,Förbättrade åtgärder när barn misstänks för br...,2022,,,sou,sou,,1,,1,,276,{'titel': 'Förbättrade åtgärder när barn misst...,sou 2022 1 Förbättrade åtgärder när barn miss...,Förbättrade åtgärder när barn misstänks för br...,sou 2022 1 Förbättrade åtgärder när barn miss...,None,"{'fil': [{'typ': 'pdf', 'namn': 'sou_2022__1.p...",dokument,,,,,,,,,,,,,None,Statens offentliga utredningar,Statens offentliga utredningar
3,4,rdwebb,dokument,2022-05-16,HAB323,None,,None,,,2022-05-16,2022-05-20 10:23:10,,,,HAB323,,//data.riksdagen.se/dokument/HAB323.text,//data.riksdagen.se/dokument/HAB323.html,,,,,,,En oavvislig ersättningsrätt?,2022,,,sou,sou,,23,,23,,276,"{'titel': 'En oavvislig ersättningsrätt?', 'un...",En oavvislig ersättningsrätt Betänkande av\n U...,En oavvislig ersättningsrätt?,En oavvislig ersättningsrätt Betänkande av\n U...,None,"{'fil': [{'typ': 'pdf', 'namn': 'sou_2022__23....",dokument,,,,,,,,,,,,,None,Statens offentliga utredningar,Statens offentliga utredningar
4,5,rdwebb,dokument,2022-05-18,HAB321,None,,None,,,2022-05-18,2022-05-20 10:23:39,,,,HAB321,,//data.riksdagen.se/dokument/HAB321.text,//data.riksdagen.se/dokument/HAB321.html,,,,,,,Rätt för klimatet,2022,,,sou,sou,,21,,21,,276,"{'titel': 'Rätt för klimatet', 'undertitel': '...",sou 2022 21 Rätt för klimatet\n Slutbetänkand...,Rätt för klimatet,sou 2022 21 Rätt för klimatet\n Slutbetänkand...,None,"{'fil': [{'typ': 'pdf', 'namn': 'sou_2022__21....",dokument,,,,,,,,,,,,,None,Statens offentliga utredningar,Statens offentliga utredningar


In [5]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 19
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   traff              180 non-null    object
 1   domain             180 non-null    object
 2   database           180 non-null    object
 3   datum              180 non-null    object
 4   id                 180 non-null    object
 5   rdrest             0 non-null      object
 6   slutdatum          180 non-null    object
 7   rddata             0 non-null      object
 8   plats              180 non-null    object
 9   klockslag          180 non-null    object
 10  publicerad         180 non-null    object
 11  systemdatum        180 non-null    object
 12  undertitel         180 non-null    object
 13  kalla              180 non-null    object
 14  kall_id            180 non-null    object
 15  dok_id             180 non-null    object
 16  dokumentformat     180 non-null    object
 17

### Call SDGMapper

In [6]:

for index, row in dftot.iterrows():
    urljson = "http://data.riksdagen.se/dokument/"+ row['id'] +".json"
    urlhtml = "http://data.riksdagen.se/dokument/"+ row['id']
    urldoc = "https://www.riksdagen.se/sv/dokument-lagar/dokument/_"+ row['id'] 
    print("\n",row['id'])
    print("\tjson url:",urljson)
    print("\tdoc url:",urldoc)
    print("\thtml url:",urlhtml)   
    
    print("\t",RenderHyperlink("",urldoc))
    SDGMapperHTML(urlhtml)


 HAB33
	json url: http://data.riksdagen.se/dokument/HAB33.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_HAB33
	html url: http://data.riksdagen.se/dokument/HAB33
	 


Length text:	 654393
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479898024092<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>
new error lower size  654393

 HAB32
	json url: http://data.riksdagen.se/dokument/HAB32.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_HAB32
	html url: http://data.riksdagen.se/dokument/HAB32
	 


Length text:	 941094
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479898030420<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 HAB31
	json url: http://data.riksdagen.se/dokument/HAB31.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_HAB31
	html url: http://data.riksdagen.se/dokument/HAB31
	 


Length text:	 755541
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695481543693373<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 HAB323
	json url: http://data.riksdagen.se/dokument/HAB323.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_HAB323
	html url: http://data.riksdagen.se/dokument/HAB323
	 


Length text:	 481247
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695481543707117<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>
new error lower size  481247

 HAB321
	json url: http://data.riksdagen.se/dokument/HAB321.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_HAB321
	html url: http://data.riksdagen.se/dokument/HAB321
	 

...


Length text:	 959583
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479899563308<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 H8B360
	json url: http://data.riksdagen.se/dokument/H8B360.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_H8B360
	html url: http://data.riksdagen.se/dokument/H8B360
	 


Length text:	 742020
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479899576244<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 H8B310
	json url: http://data.riksdagen.se/dokument/H8B310.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_H8B310
	html url: http://data.riksdagen.se/dokument/H8B310
	 


Length text:	 622538
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479899583828<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 H8B314
	json url: http://data.riksdagen.se/dokument/H8B314.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_H8B314
	html url: http://data.riksdagen.se/dokument/H8B314
	 


Length text:	 1425644
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479899591916<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>

 H8B312
	json url: http://data.riksdagen.se/dokument/H8B312.json
	doc url: https://www.riksdagen.se/sv/dokument-lagar/dokument/_H8B312
	html url: http://data.riksdagen.se/dokument/H8B312
	 


Length text:	 1226077
<Response [403]>
Exception:	 Expecting value: line 1 column 1 (char 0)
<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please contact for more information jrc-waf[at]ec.europa.eu.<br><br>Your support ID is: 484695479899601892<br><br><a href='javascript:history.back();'>[Go Back]</a></body></html>


In [7]:
print("Upper/lower limit:\t",tracksizeProblem,tracksizeOk)
print("Biggest SOU document in the database = ",trackbiggestSOUdocument)


Upper/lower limit:	 481247 471340
Biggest SOU document in the database =  1862422


In [8]:
print("End run: ", datetime.now())

End run:  2022-06-21 13:10:53.657940
